In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.tokenize import word_tokenize
from scipy import stats
from ast import literal_eval
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
# Tải dữ liệu từ nltk
nltk.download('punkt')
import warnings; warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HELLO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Đọc dữ liệu

In [5]:
import pandas as pd
movie_data = pd.read_csv(r"D:\projects\Recommendation_Movie\Data_csv\Model.csv")


In [6]:
movie_data.head()

,title,view,vote,describe,movie_genre,nation,follower,rating,season,combined_text
0,tiểu lam và tiểu lục 3,8681,1.6,tiểu_lam tiểu_lục,anime_bộ fantasy drama comedy,trung_quốc,7,tuổi_teen,2023.0,tiểu_lam tiểu_lục anime_bộ fantasy drama comed...
1,thất hầu bút lục,2495,5.0,nghìn hậu_duệ gia_tộc yinyang cảm_thấy đau_buồ...,anime_bộ fantasy adventure historical martial_...,trung_quốc,25,tuổi_teen,2023.0,nghìn hậu_duệ gia_tộc yinyang cảm_thấy đau_buồ...
2,kiếm vực phong vân,1247603,7.7,kiếm vực đại_lục xa_xưa thanh kiếm thần đại_gi...,anime_bộ fantasy adventure martial_arts action,trung_quốc,1417,tuổi_teen,2021.0,kiếm vực đại_lục xa_xưa thanh kiếm thần đại_gi...
3,ngạo thế cửu trùng thiên,182044,9.8,giữ thần_vật cửu_kiếp kiếm đại_lục_cửu trùng_t...,anime_bộ cartoon,trung_quốc,214,tuổi_teen,2023.0,giữ thần_vật cửu_kiếp kiếm đại_lục_cửu trùng_t...
4,độc bộ tiêu dao,3408484,5.3,quyến_luyến hồng_trần thần_phật tham_lam ta đà...,anime_bộ martial_arts action,trung_quốc,3219,tuổi_teen,2020.0,quyến_luyến hồng_trần thần_phật tham_lam ta đà...


Xử lý cột 'vote' để tạo cột đánh giá 'rate' mới

In [7]:
# Xử lý cột 'vote' để tạo cột đánh giá 'rate' mới
def assign_rating(vote):
    if vote >= 0 and vote < 2:
        return 1
    elif vote >= 2 and vote < 4:
        return 2
    elif vote >= 4 and vote < 6:
        return 3
    elif vote >= 6 and vote < 8:
        return 4
    elif vote >= 8 and vote <= 10:
        return 5
    else:
        return None  # Trường hợp vote không hợp lệ

# Áp dụng hàm assign_rating cho mỗi hàng trong cột 'vote'
movie_data['rate'] = movie_data['vote'].apply(assign_rating)

# Hiển thị DataFrame để kiểm tra cột 'rate' mới
movie_data.head()

,title,view,vote,describe,movie_genre,nation,follower,rating,season,combined_text,rate
0,tiểu lam và tiểu lục 3,8681,1.6,tiểu_lam tiểu_lục,anime_bộ fantasy drama comedy,trung_quốc,7,tuổi_teen,2023.0,tiểu_lam tiểu_lục anime_bộ fantasy drama comed...,1
1,thất hầu bút lục,2495,5.0,nghìn hậu_duệ gia_tộc yinyang cảm_thấy đau_buồ...,anime_bộ fantasy adventure historical martial_...,trung_quốc,25,tuổi_teen,2023.0,nghìn hậu_duệ gia_tộc yinyang cảm_thấy đau_buồ...,3
2,kiếm vực phong vân,1247603,7.7,kiếm vực đại_lục xa_xưa thanh kiếm thần đại_gi...,anime_bộ fantasy adventure martial_arts action,trung_quốc,1417,tuổi_teen,2021.0,kiếm vực đại_lục xa_xưa thanh kiếm thần đại_gi...,4
3,ngạo thế cửu trùng thiên,182044,9.8,giữ thần_vật cửu_kiếp kiếm đại_lục_cửu trùng_t...,anime_bộ cartoon,trung_quốc,214,tuổi_teen,2023.0,giữ thần_vật cửu_kiếp kiếm đại_lục_cửu trùng_t...,5
4,độc bộ tiêu dao,3408484,5.3,quyến_luyến hồng_trần thần_phật tham_lam ta đà...,anime_bộ martial_arts action,trung_quốc,3219,tuổi_teen,2020.0,quyến_luyến hồng_trần thần_phật tham_lam ta đà...,3


Tạo cột 'user_id' với giá trị ngẫu nhiên từ 1 đến 300

In [8]:
# Tạo cột 'user_id' với giá trị ngẫu nhiên từ 1 đến 300
num_users = 300
movie_data['user_id'] = np.random.randint(1, num_users + 1, size=movie_data.shape[0])

movie_data.head()

,title,view,vote,describe,movie_genre,nation,follower,rating,season,combined_text,rate,user_id
0,tiểu lam và tiểu lục 3,8681,1.6,tiểu_lam tiểu_lục,anime_bộ fantasy drama comedy,trung_quốc,7,tuổi_teen,2023.0,tiểu_lam tiểu_lục anime_bộ fantasy drama comed...,1,90
1,thất hầu bút lục,2495,5.0,nghìn hậu_duệ gia_tộc yinyang cảm_thấy đau_buồ...,anime_bộ fantasy adventure historical martial_...,trung_quốc,25,tuổi_teen,2023.0,nghìn hậu_duệ gia_tộc yinyang cảm_thấy đau_buồ...,3,248
2,kiếm vực phong vân,1247603,7.7,kiếm vực đại_lục xa_xưa thanh kiếm thần đại_gi...,anime_bộ fantasy adventure martial_arts action,trung_quốc,1417,tuổi_teen,2021.0,kiếm vực đại_lục xa_xưa thanh kiếm thần đại_gi...,4,60
3,ngạo thế cửu trùng thiên,182044,9.8,giữ thần_vật cửu_kiếp kiếm đại_lục_cửu trùng_t...,anime_bộ cartoon,trung_quốc,214,tuổi_teen,2023.0,giữ thần_vật cửu_kiếp kiếm đại_lục_cửu trùng_t...,5,99
4,độc bộ tiêu dao,3408484,5.3,quyến_luyến hồng_trần thần_phật tham_lam ta đà...,anime_bộ martial_arts action,trung_quốc,3219,tuổi_teen,2020.0,quyến_luyến hồng_trần thần_phật tham_lam ta đà...,3,201


Loại bỏ ký tự số cho cột 'combined_text'

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler

# Loại bỏ ký tự số
movie_data['combined_text'] = movie_data['combined_text'].str.replace(r'\d+', '')


Trích xuất văn bản

In [10]:
# Vector hóa văn bản sử dụng TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(movie_data['combined_text'])

# Tạo ma trận tương tác cho collaborative filtering
interaction_matrix = movie_data.pivot(index='user_id', columns='title', values='rate').fillna(0)

# Chuẩn hóa ma trận tương tác
scaler = MinMaxScaler()
interaction_matrix_scaled = scaler.fit_transform(interaction_matrix)
interaction_matrix_scaled_df = pd.DataFrame(interaction_matrix_scaled, index=interaction_matrix.index, columns=interaction_matrix.columns)


Content-based filtering

In [11]:
# Tính toán độ tương đồng cosine
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Tạo một mapping từ tiêu đề phim đến chỉ số của chúng
indices = pd.Series(movie_data.index, index=movie_data['title']).drop_duplicates()

def get_similar_movies(title, n=5):
    # Kiểm tra xem tiêu đề có tồn tại không
    if title not in indices:
        return f"No movie found with title '{title}'."

    idx = indices[title]

    # Lấy điểm tương đồng của tất cả các phim với phim này
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sắp xếp các phim dựa trên điểm tương đồng
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Lấy điểm của N phim có điểm tương đồng cao nhất
    sim_scores = sim_scores[1:n+1]

    movie_indices = [i[0] for i in sim_scores]

    return movie_data['title'].iloc[movie_indices]

# Ví dụ sử dụng
similar_movies = get_similar_movies("tiểu lam và tiểu lục 3", 5)
print(similar_movies)


113         tiểu lam và tiểu lục 2
1747          tiểu lục và tiểu lam
178     nhất niệm vĩnh hằng phần 2
74        thiên quan tứ phúc mùa 2
246             tây hành kỷ phần 4
Name: title, dtype: object


Collaborative filtering

In [12]:
# Collaborative filtering
from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate

# Đầu tiên, chuyển ma trận tương tác thành định dạng mà thư viện surprise có thể sử dụng
# Chúng ta cần 'user_id', 'title' và 'rate' từ movie_data để tạo tập dữ liệu
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(movie_data[['user_id', 'title', 'rate']], reader)

# Sử dụng SVD, một phương pháp matrix factorization
svd = SVD()

# Thực hiện cross-validation hoặc tập dữ liệu có thể được chia thành tập huấn luyện và kiểm tra
# Ví dụ: cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
trainset = data.build_full_trainset()
svd.fit(trainset)

# Dự đoán sở thích của người dùng cho một phim cụ thể
# Giả sử chúng ta muốn dự đoán đánh giá của người dùng với user_id = 1 cho phim có title là 'tiểu lam và tiểu lục 3'
user_id = 1
title = 'tiểu lam và tiểu lục 3'
predicted_rating = svd.predict(user_id, title).est

print(f'The estimated rating for {title} by user {user_id} is {predicted_rating}')


The estimated rating for tiểu lam và tiểu lục 3 by user 1 is 4.169901931062342


Hybrid_recommendations

In [13]:
# Định nghĩa một hàm để kết hợp độ tương đồng content-based và collaborative filtering
def hybrid_recommendations(user_id, title, n=5):
    # Lấy chỉ số của phim từ tiêu đề
    idx = indices[title]
    
    # Độ tương đồng content-based
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Dự đoán đánh giá collaborative filtering
    svd_predictions = [svd.predict(user_id, movie_data['title'].iloc[i[0]]).est for i in sim_scores]
    
    # Kết hợp độ tương đồng và dự đoán đánh giá
    hybrid_scores = [(i[0], (i[1] + svd_predictions[idx]) / 2) for idx, i in enumerate(sim_scores)]
    
    # Sắp xếp các phim dựa trên điểm hybrid
    hybrid_scores = sorted(hybrid_scores, key=lambda x: x[1], reverse=True)
    
    # Lấy top N kết quả
    hybrid_indices = [i[0] for i in hybrid_scores[1:n+1]]
    
    # Trả về danh sách tiêu đề phim đề xuất
    return movie_data['title'].iloc[hybrid_indices]

# Ví dụ đề xuất phim cho người dùng với ID 1 và phim 'tiểu lam và tiểu lục 3'
recommendations = hybrid_recommendations(1, 'tiểu lam và tiểu lục 3')
print(recommendations)



0       tiểu lam và tiểu lục 3
980       elsword: el-ui yeoin
1747      tiểu lục và tiểu lam
610        strike the blood iv
579             sử ma của zero
Name: title, dtype: object


Model Content-based filtering

In [2]:
# Model Content-based filtering
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

class MovieRecommenderTitle:
    def __init__(self, movie_data_path):
        # Load the movie data from the CSV file
        self.movie_data = pd.read_csv(movie_data_path)

        # Ensure all movie titles are in lower case
        self.movie_data['title'] = self.movie_data['title'].str.lower()

        # Create a TF-IDF matrix
        self.tfidf_vectorizer = TfidfVectorizer()
        self.tfidf_matrix = self.tfidf_vectorizer.fit_transform(self.movie_data['combined_text'])

        # Calculate cosine similarity
        self.cosine_sim = cosine_similarity(self.tfidf_matrix, self.tfidf_matrix)

        # Create a mapping from movie titles to their indices
        self.indices = pd.Series(self.movie_data.index, index=self.movie_data['title']).drop_duplicates()

    def get_similar_movies(self, title, n=10):
        # Convert the input title to lower case
        title = title.lower()

        # Check if the title exists
        if title not in self.indices:
            return f"No movie found with title '{title}'."

        idx = self.indices[title]

        # Get the similarity scores of all movies with this movie
        sim_scores = list(enumerate(self.cosine_sim[idx]))

        # Sort the movies based on similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Get the scores of the top N similar movies
        sim_scores = sim_scores[1:n+1]

        movie_indices = [i[0] for i in sim_scores]

        return self.movie_data['title'].iloc[movie_indices]


Lưu mô hình lại trong file Model_Contentbased.pkl

In [15]:
# Lưu mô hình lại trong file Model_Contentbased.pkl
import pickle
# Tạo một đối tượng MovieRecommender
recommender = MovieRecommenderTitle(movie_data_path=r"D:\projects\Recommendation_Movie\Data_csv\Model.csv")

# Lưu mô hình vào tệp model.pkl
with open('Model_Contentbased.pkl', 'wb') as model_file:
    pickle.dump(recommender, model_file)

In [16]:
# Load Model_Contentbased.pkl
import pickle

# Load mô hình từ file pickle
with open('Model_Contentbased.pkl', 'rb') as model_file:
    recommender = pickle.load(model_file)

# Sử dụng recommender
result = recommender.get_similar_movies("tiểu lục và tiểu lam")
print(result)

113                              tiểu lam và tiểu lục 2
0                                tiểu lam và tiểu lục 3
426                        bleach: huyết chiến ngàn năm
3506                                 sengoku collection
2900        bishoujo senshi sailor moon eternal movie 2
2043                   thế giới phép thuật bản đặc biệt
1323            mirai shounen conan - cậu bé thông minh
270         tiểu kịch trường - người đại diện thời gian
1042                                         ani*kuri15
1794    free!: take your marks: character butai aisatsu
Name: title, dtype: object


Tích hợp thêm với fuzzywuzzy

In [2]:
from fuzzywuzzy import process

class MovieRecommenderTitleFuzzy:
    def __init__(self, movie_data_path):
        # Load the movie data from the CSV file
        self.movie_data = pd.read_csv(movie_data_path)

        # Ensure all movie titles are in lower case
        self.movie_data['title'] = self.movie_data['title'].str.lower()

        # Create a TF-IDF matrix
        self.tfidf_vectorizer = TfidfVectorizer()
        self.tfidf_matrix = self.tfidf_vectorizer.fit_transform(self.movie_data['combined_text'])

        # Calculate cosine similarity
        self.cosine_sim = cosine_similarity(self.tfidf_matrix, self.tfidf_matrix)

        # Create a mapping from movie titles to their indices
        self.indices = pd.Series(self.movie_data.index, index=self.movie_data['title']).drop_duplicates()

    def get_similar_movies(self, title, n=10):
        # Tìm tiêu đề phim gần đúng nhất với từ khóa nhập vào
        title = title.lower()
        closest_title = process.extractOne(title, self.indices.index, scorer=process.fuzz.token_sort_ratio)

        # Nếu không tìm thấy tiêu đề gần đúng
        if not closest_title:
            return f"No close match found for '{title}'."

        closest_title = closest_title[0]

        # Lấy chỉ số của tiêu đề gần đúng nhất
        idx = self.indices[closest_title]

        # Tính toán độ tương đồng và lấy các phim tương tự
        sim_scores = list(enumerate(self.cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        top_similar_indices = [i[0] for i in sim_scores[1:n+1]]

        return self.movie_data['title'].iloc[top_similar_indices].tolist()



Lưu mô hình lại trong file ModelFuzzy_Contentbased.pkl

In [18]:
# Lưu mô hình lại trong file ModelFuzzy_Contentbased.pkl
import pickle
# Tạo một đối tượng MovieRecommender
recommender = MovieRecommenderTitleFuzzy(movie_data_path=r"D:\projects\Recommendation_Movie\Data_csv\Model.csv")

# Lưu mô hình vào tệp model.pkl
with open('ModelFuzzy_Contentbased.pkl', 'wb') as model_file:
    pickle.dump(recommender, model_file)

In [19]:
# Load ModelFuzzy_Contentbased.pkl
import pickle

# Load mô hình từ file pickle
with open('ModelFuzzy_Contentbased.pkl', 'rb') as model_file:
    recommender = pickle.load(model_file)

# Sử dụng recommender
result = recommender.get_similar_movies("doremon")
print(result)

['chuyện về tamako', 'yume-iro pâtissière', 'shokugeki no souma', 'kiếp chuyển sinh ngọt ngào', 'mayoi neko overrun!', 'bonjour♪koiaji pâtisserie', 'sacred seven', 'date a live ii', 'nekopara', 'denki-gai no honya-san']


Model BERT

In [3]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

class MovieRecommender:
    def __init__(self, model_name='bert-base-uncased', movie_data_path=r"D:\projects\Recomendation_Movie\Data_csv\Model.csv"):
        # Khởi tạo tokenizer và model BERT
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertModel.from_pretrained(model_name)
        
        # Đọc dữ liệu phim
        self.movie_data = pd.read_csv(movie_data_path)
        
        # Loại bỏ các hàng có giá trị 'nan' trong 'combined_text'
        self.movie_data.dropna(subset=['combined_text'], inplace=True)

        # Mã hóa mô tả phim từ cột 'combined_text'
        self.movie_embeddings = self._encode_movie_texts_in_batches()

    def _encode_text(self, text):
        inputs = self.tokenizer.encode(text, return_tensors='pt', max_length=128, truncation=True)
        with torch.no_grad():
            outputs = self.model(inputs)[0]
        return outputs.mean(dim=1)

    def _encode_movie_texts_in_batches(self, batch_size=10):
        embeddings = []
        for i in range(0, len(self.movie_data), batch_size):
            batch_texts = self.movie_data['combined_text'][i:i + batch_size]
            batch_embeddings = [self._encode_text(text) for text in batch_texts]
            embeddings.extend(batch_embeddings)
        return embeddings

    def recommend_movies(self, search_query):
        # Mã hóa truy vấn tìm kiếm và tính toán độ tương tự
        search_embedding = self._encode_text(search_query).squeeze()
        movie_embeddings = torch.stack(self.movie_embeddings).squeeze(dim=1)

        similarities = cosine_similarity(search_embedding.unsqueeze(0), movie_embeddings)
        sorted_indices = similarities.argsort()[0][::-1]
        recommended_titles = self.movie_data.iloc[sorted_indices]['title'].values
        return recommended_titles[:10]


Lưu mô hình lại trong file ModelBERT.pkl

In [21]:
import pickle
# Tạo một đối tượng MovieRecommender
recommender = MovieRecommender(movie_data_path=r"D:\projects\Recommendation_Movie\Data_csv\Model.csv")

# Lưu mô hình vào tệp model.pkl
with open('ModelBERT.pkl', 'wb') as model_file:
    pickle.dump(recommender, model_file)

Dùng model đã lưu

In [4]:
# Load ModelBERT.pkl
import pickle

# Load mô hình từ file pickle
with open('ModelBERT.pkl', 'rb') as model_file:
    recommender = pickle.load(model_file)

# Sử dụng recommender
result = recommender.recommend_movies("doraemon")
print(result)

['doraemon movie 03: nobita no daimakyou'
 'doraemon movie 09: nobita no parallel saiyuuki'
 'doraemon movie 02: nobita no uchuu kaitakushi' 'fairy tail ova'
 'doraemon movie 04: nobita no kaitei kiganjou'
 'chain chronicle: haecceitas no hikari movie'
 'senjou no valkyria: senval theater'
 'toaru kagaku no railgun s: daiji na koto wa zenbu sentou ni osowatta'
 'doraemon movie 06: nobita no little star wars'
 'chúng tôi không thể học ova']


In [5]:
import pickle
from flask import Flask, render_template, request

app = Flask(__name__)

# Load mô hình từ file pickle
with open('ModelBERT.pkl', 'rb') as model_file:
    recommender = pickle.load(model_file)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/recommend', methods=['POST'])
def recommend():
    # Nhận dữ liệu nhập từ form
    movie_title = request.form['movie_title']
    
    # Sử dụng recommender
    result = recommender.recommend_movies(movie_title)
    
    # Trả về kết quả
    return render_template('result.html', result=result)

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [19/Dec/2023 09:31:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Dec/2023 09:32:00] "POST /recommend HTTP/1.1" 200 -


In [ ]:
# Fuzzy
ModelFuzzy_Contentbased
get_similar_movies

#BERT
ModelBERT
recommend_movies

#Contentbase
Model_Contentbased
get_similar_movies